In [1]:
import cv2
import numpy as np
import imageio as io
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize,StandardScaler
import numpy as np
import os
import random
from sklearn.model_selection import train_test_split
import skimage.io as io
from skimage.feature import local_binary_pattern
from skimage import color
from operator import eq
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import pandas as pd

In [2]:
random_seed = 0  
random.seed(random_seed)
np.random.seed(random_seed)

In [3]:
def extract_hog_features(img):

    img=color.rgb2gray(img)
    img = img.astype(int)

    target_img_size = (78, 78)
    img = cv2.resize(img.astype('uint8'), target_img_size)
    win_size = (64, 64)
    cell_size = (8, 8)
    block_size_in_cells = (2, 2)

    block_size = (block_size_in_cells[1] * cell_size[1],
                  block_size_in_cells[0] * cell_size[0])
    block_stride = (cell_size[1], cell_size[0])
    nbins = 15  # Number of orientation bins
    hog = cv2.HOGDescriptor(win_size, block_size,
                            block_stride, cell_size, nbins)
    h = hog.compute(img)
    h = h.flatten()
    return h.flatten()

In [4]:
# img  = io.imread('S010_006_00000013.png')
# gray=color.rgb2gray(img)
# f=extract_hog_features(gray)
# print(np.array(f).shape )


In [5]:
# def extract_features(img):
#     return extract_hog_features(img) 

In [6]:
def load_dataset():
    features = []
    labels = []
    for i, fn in enumerate(img_filenames):
            label = fn.split('.')[0]
            path = os.path.join(path_to_dataset, fn)
            subpath=os.listdir(path)
            for k, d in enumerate(subpath):             
                labels.append(label)
                imgpath = os.path.join(path, d)
                img = io.imread(imgpath)
                gray=color.rgb2gray(img)
                features.append(extract_hog_features(gray))
                if k > 0 and k % 100== 0:
                    print("[INFO] processed {}/{}".format(k, len(subpath)))
    return features, labels            

In [7]:
def split_data(features,labels,precentage):
    train_features, test_features, train_labels, test_labels = train_test_split(
        features, labels, test_size=0.2, random_state=random_seed)
    return train_features, test_features, train_labels, test_labels

In [8]:
def KNN_classifier(kcount,train_features, train_labels):
    classifier = KNeighborsClassifier(n_neighbors=kcount)
    classifier.fit(list(train_features),list(train_labels) )
    return classifier

In [9]:
# path_to_dataset =r'D:\Download\jaffedbase'
# path_to_dataset =r'E:\datasetemotion\val\val'
path_to_dataset =r'E:\emotion\archive\ck\CK+48'
img_filenames = os.listdir(path_to_dataset)

In [10]:
img_filenames = os.listdir(path_to_dataset)
print(img_filenames)

['anger', 'contempt', 'disgust', 'fear', 'happy', 'sadness', 'surprise']


In [11]:
random_seed = 0  
random.seed(random_seed)
np.random.seed(random_seed)

In [12]:
print('Loading dataset. This will take time ...')
features, labels = load_dataset()
print('Finished loading dataset.')

Loading dataset. This will take time ...
[INFO] processed 100/135
[INFO] processed 100/177
[INFO] processed 100/207
[INFO] processed 200/207
[INFO] processed 100/249
[INFO] processed 200/249
Finished loading dataset.


In [13]:
fg=features
lg=labels

In [14]:
train_features, test_features, train_labels, test_labels=split_data(fg,lg,0.2)


In [15]:
print(np.array(train_features).shape)

(784, 11760)


In [16]:
from sklearn.decomposition import PCA

pca = PCA(n_components=0.7)
X_train1 = pca.fit_transform(train_features)
X_test1 = pca.transform(test_features)


In [17]:
train_features=X_train1
test_features=X_test1

In [18]:
from sklearn.metrics.pairwise import rbf_kernel
le = preprocessing.LabelEncoder()
random_seed = 0  
random.seed(random_seed)
np.random.seed(random_seed)
sigma=0.0008
c=64
reg=100
err=1e-4
def kernel(X, Y=None,g=None):
    k=rbf_kernel(X, Y, gamma=g)
    return k
def KELM_Beta(sigma,c,x,y):
    le.fit(y)
    y = le.transform(y)
    emotion = len(np.unique(y))
    One_Hot_Encoding=np.zeros((emotion,emotion), np.uint8)
    for em in range (emotion):
        One_Hot_Encoding[em][em]=1
    T = One_Hot_Encoding[y, :]    
    N,d=x.shape
    Omega=kernel(x,None,sigma)
    I=np.eye(N)
    Beta = np.linalg.inv((I /c) + Omega).dot(T)    
    return Beta,One_Hot_Encoding
def KELM_Output(sigma,x,x_test,Beta,One_Hot_Encoding):
    hx=kernel(x_test,x,sigma)
    fx= hx.dot(Beta)
#     print(np.array(fx).shape)
    p=np.argmax(fx,axis=1)
    return p
def acc(y_pred,test_labels):
    le = preprocessing.LabelEncoder()
    yt = le.fit_transform(y_pred)    
    rt = le.fit_transform(test_labels)
    eqw = sum(map(eq, list(rt), list(y_pred)))
    size=len(rt)
    acc=(eqw/size)*100
    return acc
Beta,One_Hot_Encoding=KELM_Beta(sigma,c,np.array(train_features),np.array(train_labels))
p1=KELM_Output(sigma,np.array(train_features),np.array(test_features),Beta,One_Hot_Encoding)
print("test",acc(p1,test_labels))

p2=KELM_Output(sigma,np.array(train_features),np.array(train_features),Beta,One_Hot_Encoding)
print("train",acc(p2,train_labels))

test 96.44670050761421
train 99.23469387755102


In [19]:
print(np.array(X_test1).shape)

(197, 64)


In [20]:
#SVM

In [21]:
from sklearn.svm import SVC
k='rbf'
gamma = 'scale'
c=200
# clf = SVC(C=c,kernel=k,gamma=gamma)
clf = SVC()

clf.fit(X_train1, train_labels)
ypred=clf.predict(X_test1)
print ('Test score', clf.score(X_test1, test_labels)*100 )
print ('Train score', clf.score(X_train1, train_labels)*100 )

# Test score 98.47715736040608
# Train score 99.74489795918367


Test score 98.47715736040608
Train score 99.74489795918367


In [22]:
# import joblib
# joblib.dump(clf, 'PCASVM.pkl')


In [23]:
# print(test_labels)
print(accuracy_score(test_labels, ypred))
print(classification_report(test_labels, ypred))
print(confusion_matrix(test_labels, ypred))

0.9847715736040609
              precision    recall  f1-score   support

       anger       1.00      1.00      1.00        20
    contempt       0.75      1.00      0.86         6
     disgust       1.00      1.00      1.00        41
        fear       1.00      0.93      0.97        15
       happy       1.00      1.00      1.00        39
     sadness       1.00      0.91      0.95        22
    surprise       0.98      1.00      0.99        54

    accuracy                           0.98       197
   macro avg       0.96      0.98      0.97       197
weighted avg       0.99      0.98      0.99       197

[[20  0  0  0  0  0  0]
 [ 0  6  0  0  0  0  0]
 [ 0  0 41  0  0  0  0]
 [ 0  0  0 14  0  0  1]
 [ 0  0  0  0 39  0  0]
 [ 0  2  0  0  0 20  0]
 [ 0  0  0  0  0  0 54]]


In [24]:
#Test(New img)

In [25]:
# from sklearn.decomposition import PCA
# # img = io.imread('MK.SU1.122.tiff')
# img = io.imread('S010_006_00000013.png')
# # /happy
# # img = io.imread('MK.HA1.116.tiff')
# # /angry
# # img = io.imread('KA.AN3.41.tiff')
# # /sad
# # img = io.imread('KA.SA1.33.tiff')
# # img = io.imread('data.png')

# # img  = io.imread('KA.AN2.40.tiff')
# # img  = io.imread('MK.HA1.116.tiff')
# # img  = io.imread('UY.DI2.150.tiff')

# # img = io.imread('p1.jpg')

# f=extract_features(img)
# fp= pca.transform([f])
# # print(f)
# # print(fp)

# print ('pred label', clf.predict(fp) )

In [26]:
#KNN

In [27]:
classifier=KNN_classifier(3,X_train1, train_labels)


In [28]:
y_pred = classifier.predict(X_test1 )

In [29]:
accuracy = classifier.score(X_test1, test_labels)
print('Test accuracy:', accuracy*100, '%')
accuracy = classifier.score(X_train1, train_labels)
print('Train accuracy:', accuracy*100, '%')
# Test accuracy: 81.7258883248731 %
# Train accuracy: 97.95918367346938 %
    

Test accuracy: 81.21827411167513 %
Train accuracy: 97.83163265306123 %


In [30]:
# import joblib
# joblib.dump(classifier, 'PCAKNN.pkl')


In [31]:
# print(test_labels)
print(accuracy_score(test_labels, y_pred))
print(classification_report(test_labels, y_pred))
print(confusion_matrix(test_labels, y_pred))

0.8121827411167513
              precision    recall  f1-score   support

       anger       0.75      0.90      0.82        20
    contempt       0.75      1.00      0.86         6
     disgust       0.88      0.73      0.80        41
        fear       0.76      0.87      0.81        15
       happy       0.92      0.85      0.88        39
     sadness       0.60      0.55      0.57        22
    surprise       0.83      0.89      0.86        54

    accuracy                           0.81       197
   macro avg       0.78      0.83      0.80       197
weighted avg       0.82      0.81      0.81       197

[[18  0  0  0  0  2  0]
 [ 0  6  0  0  0  0  0]
 [ 2  0 30  2  3  2  2]
 [ 0  0  0 13  0  2  0]
 [ 2  0  2  0 33  0  2]
 [ 0  2  2  0  0 12  6]
 [ 2  0  0  2  0  2 48]]


In [33]:
# # img = io.imread('im.JPG')
# # img = io.imread('S010_006_00000013.png')
# # img = io.imread('MK.SU1.122.tiff')
# # img = io.imread('S010_006_00000013.png')
# # /happy
# # img = io.imread('MK.HA1.116.tiff')
# # /angry
# # img = io.imread('KA.AN3.41.tiff')
# # /sad
# # img = io.imread('KA.SA1.33.tiff')
# # img = io.imread('data.png')

# # img  = io.imread('KA.AN2.40.tiff')
# # img  = io.imread('MK.HA1.116.tiff')
# # img  = io.imread('UY.DI2.150.tiff')

# img = io.imread('S010_006_00000013.png')
# # /happy
# # img = io.imread('MK.HA1.116.tiff')
# # /angry
# # img = io.imread('KA.AN3.41.tiff')
# # /sad
# # img = io.imread('KA.SA1.33.tiff')
# # img = io.imread('data.png')

# # img  = io.imread('KA.AN2.40.tiff')
# # img  = io.imread('MK.HA1.116.tiff')
# # img  = io.imread('UY.DI2.150.tiff')

# # img = io.imread('p1.jpg')

# f=extract_features(img)
# fp= pca.transform([f])
# # print(f)
# # print(fp)

# print ('pred label', classifier.predict(fp) )

In [34]:
# import joblib

# train_features, test_features, train_labels, test_labels=split_data(fg,lg,0.2)
# # from sklearn.decomposition import PCA

# # pca = PCA(n_components=0.7)
# # X_train1 = pca.fit_transform(train_features)
# # X_test1 = pca.transform(test_features)
# # # n_components=1000 must be between 0 and min(n_samples, n_features)=900 with svd_solver='full'

# PCASVM_joblib = joblib.load('PCASVM.pkl')
# # Use the loaded model to make predictions
# g=PCASVM_joblib.predict(test_features)
# print(g)